# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
# Do all imports and installs here
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [ ]:
# Read in the data here
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [ ]:
df.head()

In [83]:
import findspark
findspark.init('C:\spark')
               
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
#df_visits = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [98]:
#write to parquet
#df_visits.write.parquet("sas_data")
df_visits=spark.read.parquet("data/immigration_data")
df_demo = spark.read.csv("data/us-cities-demographics.csv", sep=";", header=True)
df_airports = spark.read.csv("data/airport-codes_csv.csv", header=True)

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [117]:
# Performing cleaning tasks here
# df_visits.show(3)
#df_visits.describe()
df_visits.count()

3096313

In [150]:
# parse SAS_labels_descriptions file for valid country codes
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType
f = open("data/I94_SAS_Labels_Descriptions.SAS","r")
lines = f.readlines()
countries=[]
for line in lines[9:245]:
    line_split = line.replace("'", "").split('=')
    countries.append({"country_code":int(line_split[0].strip()), "country_name":line_split[1].strip()})

schema = StructType([
    StructField('country_code', IntegerType()),
    StructField('country_name', StringType())
])

df_countries = spark.createDataFrame(countries, schema)
df_countries.show(4)
df_countries.createOrReplaceTempView("countries")
df_countries.count()

+------------+--------------------+
|country_code|        country_name|
+------------+--------------------+
|         582|MEXICO Air Sea, a...|
|         236|         AFGHANISTAN|
|         101|             ALBANIA|
|         316|             ALGERIA|
+------------+--------------------+
only showing top 4 rows



236

In [57]:
# parse SAS_labels_descriptions file for valid airport code
f = open("data/I94_SAS_Labels_Descriptions.SAS", "r")
lines = f.readlines()
airports=[]
for line in lines[302:893]:
    line_split = line.replace("'", "").split('=')
    airports.append({"airport_code":line_split[0].strip(), "airport_name":line_split[1].strip()})

schema = StructType([
    StructField('airport_code', StringType()),
    StructField('airport_name', StringType())
])

df_airport_codes = spark.createDataFrame(airports, schema)
df_airport_codes.show(4)
df_airport_codes.createOrReplaceTempView("lookup_airport_codes")
df_airport_codes.count()

+------------+--------------------+
|airport_code|        airport_name|
+------------+--------------------+
|         ALC|           ALCAN, AK|
|         ANC|       ANCHORAGE, AK|
|         BAR|BAKER AAF - BAKER...|
|         DAC|   DALTONS CACHE, AK|
+------------+--------------------+
only showing top 4 rows



591

In [58]:
df_visits.createOrReplaceTempView("visits")

spark.sql("""select cicid,i94cit,i94res,i94port from visits 
where i94port not in (select airport_code from lookup_airport_codes)""").count()

spark.sql("""select cicid,i94cit,i94res,i94port from visits 
where i94cit not in (select country_code from countries)""").count()

394068

In [99]:
# rename columns(meaningful), fix column data types, convert arr/dep date to datetime, remove invalid airports,countries
df_visits.createOrReplaceTempView("visits")

df_visits_format = spark.sql("""SELECT cast(cicid as int) id, cast(i94cit as int) country_citizenship, 
cast(i94res as int) country_residence, i94port port, date_add('1960-01-01', cast(arrdate as int)) arrival_date, 
date_add('1960-01-01', cast(depdate as int)) departure_date, I94addr state, cast(i94bir as int) age,gender, 
cast(i94visa as int) visa, airline FROM visits 
""")

df_visits_format.createOrReplaceTempView("visits")

df_visits_clean = spark.sql("""SELECT id, country_citizenship, country_residence, port, arrival_date, 
departure_date, state, age,gender, visa,airline FROM visits 
where port in (select airport_code from lookup_airport_codes)
and country_residence in (select country_code from countries)
and country_citizenship in (select country_code from countries)
""")

df_visits_clean.createOrReplaceTempView("visits")
df_visits_clean.show(3)

+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+---------+-------+
|     id|country_citizenship|country_residence|port|arrival_date|departure_date|state|age|gender|visa_type|airline|
+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+---------+-------+
|4557229|                392|              392| FMY|  2016-04-24|    2016-04-29|   NY| 48|     M|        2|     AF|
|3908787|                392|              392| FMY|  2016-04-21|          null|   PA|  3|  null|        2|     AT|
|5515827|                392|              392| DET|  2016-04-29|    2016-05-05|   PA| 64|     M|        1|     DL|
+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+---------+-------+
only showing top 3 rows



In [96]:
df_visits_clean.count()

2698166

In [143]:
# Discard non-US airports
df_airports.createOrReplaceTempView("airports")

df_airports_clean = spark.sql("""select name, iso_country, iso_region, municipality, iata_code, local_code From airports 
where iso_country='US' and local_code is not null
""")

df_airports_clean.createOrReplaceTempView("airports")

df_airports_clean.show(3)
df_airports_clean.count()

+--------------------+-----------+----------+------------+---------+----------+
|                name|iso_country|iso_region|municipality|iata_code|local_code|
+--------------------+-----------+----------+------------+---------+----------+
|   Total Rf Heliport|         US|     US-PA|    Bensalem|     null|       00A|
|Aero B Ranch Airport|         US|     US-KS|       Leoti|     null|      00AA|
|        Lowell Field|         US|     US-AK|Anchor Point|     null|      00AK|
+--------------------+-----------+----------+------------+---------+----------+
only showing top 3 rows



21236

In [70]:
df_demo.createOrReplaceTempView("demo")
#df_demo.describe()

# spark.sql("SELECT * FROM demo where city='Quincy' and state='Massachusetts'").show()

df_demo_format = spark.sql("""select distinct city,`State Code` state,cast(`Median Age` as float) median_age, 
cast(`Male Population` as int) male_population,cast(`Female Population` as int) female_population, 
cast(`Total Population` as int) total_population, cast(`Foreign-born` as int) foreign_born from demo where state is not null
""")

df_demo_format.show(4)
df_demo_format.count()
df_demo_format.describe()

df_demo_format.createOrReplaceTempView("demo")
spark.sql("select * from demo where median_age>35 ").show()

+-----------+-----+----------+---------------+-----------------+----------------+------------+
|       city|state|median_age|male_population|female_population|total_population|foreign_born|
+-----------+-----+----------+---------------+-----------------+----------------+------------+
|  Harlingen|   TX|      30.1|          32404|            33365|           65769|       10391|
|Brownsville|   TX|      30.6|          87689|            96199|          183888|       53301|
|    Sunrise|   FL|      37.7|          41471|            51235|           92706|       38209|
|  Vacaville|   CA|      35.8|          50091|            46703|           96794|       10577|
+-----------+-----+----------+---------------+-----------------+----------------+------------+
only showing top 4 rows

+--------------------+-----+----------+---------------+-----------------+----------------+------------+
|                city|state|median_age|male_population|female_population|total_population|foreign_born|
+------

In [106]:
# parse SAS_labels_descriptions file for state codes and names
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType
f = open("data/I94_SAS_Labels_Descriptions.SAS","r")
lines = f.readlines()
states=[]
for line in lines[981:1035]:
    line_split = line.replace("'", "").split('=')
    states.append({"code":line_split[0].strip(), "name":line_split[1].strip()})

schema = StructType([
    StructField('code', StringType()),
    StructField('name', StringType())
])

df_states = spark.createDataFrame(states, schema)
df_states.show(4)
df_states.count()
print(f"First state: {states[0]}, Last state: {states[-1]}")

+----+--------+
|code|    name|
+----+--------+
|  AL| ALABAMA|
|  AK|  ALASKA|
|  AZ| ARIZONA|
|  AR|ARKANSAS|
+----+--------+
only showing top 4 rows

First state: {'code': 'AL', 'name': 'ALABAMA'}, Last country: {'code': 'WY', 'name': 'WYOMING'}


In [108]:
# parse SAS_labels_descriptions file for visa types
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DecimalType
f = open("data/I94_SAS_Labels_Descriptions.SAS","r")
lines = f.readlines()
visa=[]
for line in lines[1046:1049]:
    line_split = line.replace("'", "").split('=')
    visa.append({"id":int(line_split[0].strip()), "visa":line_split[1].strip()})

schema = StructType([
    StructField('id', IntegerType()),
    StructField('visa', StringType())
])

df_visa = spark.createDataFrame(visa, schema)
df_visa.show(4)
df_visa.createOrReplaceTempView("visa")
df_visa.count()

+---+--------+
| id|    visa|
+---+--------+
|  1|Business|
|  2|Pleasure|
|  3| Student|
+---+--------+



3

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

In [ ]:
df_temperature.show(3)

In [ ]:
df_temperature.createOrReplaceTempView("temperature")

In [ ]:
#spark.sql("select * from temperature where dt> to_date('20000101','YYYYMMDD')").show(4)
spark.sql("select * from temperature where dt>= '2000-01-01' order by dt").show(4)
#spark.sql("select dt,count(*) from temperature group by dt order by dt desc").show(4)

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

In [ ]:
results="results/"
df_demo_format.write.mode('overwrite').parquet(results +  "dim_us_demo.parquet")

In [100]:
df_visits_clean.createOrReplaceTempView("visits")
#spark.sql("""select visits.*,'tmp'as temp from visits""").show(3)
df_visits_partition = spark.sql("""select visits.*,
extract(day from arrival_date) arrival_day,
extract(month from arrival_date) arrival_month,
extract(year from arrival_date) arrival_year from visits
""")

df_visits_partition.write.partitionBy("arrival_year","arrival_month","arrival_day").mode('overwrite').parquet(results + "fact_visits.parquet")

In [103]:
df_visits_partition.show(3)
#spark.sql("""select count(*) from visits where arrival_date=to_date('04/07/2016','MM/dd/yyyy')""").show()

+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+---------+-------+-----------+-------------+------------+
|     id|country_citizenship|country_residence|port|arrival_date|departure_date|state|age|gender|visa_type|airline|arrival_day|arrival_month|arrival_year|
+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+---------+-------+-----------+-------------+------------+
|4557229|                392|              392| FMY|  2016-04-24|    2016-04-29|   NY| 48|     M|        2|     AF|         24|            4|        2016|
|3908787|                392|              392| FMY|  2016-04-21|          null|   PA|  3|  null|        2|     AT|         21|            4|        2016|
|5515827|                392|              392| DET|  2016-04-29|    2016-05-05|   PA| 64|     M|        1|     DL|         29|            4|        2016|
+-------+-------------------+-----------------+----+------------+-----

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [154]:
# Perform quality checks here
df_visits_clean.createOrReplaceTempView("visits")
df_countries.createOrReplaceTempView("countries")
#select count(*) from visits
#spark.sql("select count(*) from visits where id is null").show()
#spark.sql("select id from visits group by id having count(*)>1").show()
#spark.sql("select count(*) from visits where port not in (select airport_code from lookup_airport_codes)").show()
#spark.sql("select count(*) from visits where country_citizenship not in (select country_code from countries)").show()
#spark.sql("select count(*) from visits where country_residence not in (select country_code from countries)").show()

df_visits_clean.show(4)

+---+
| id|
+---+
+---+

+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+---------+-------+
|     id|country_citizenship|country_residence|port|arrival_date|departure_date|state|age|gender|visa_type|airline|
+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+---------+-------+
|4557229|                392|              392| FMY|  2016-04-24|    2016-04-29|   NY| 48|     M|        2|     AF|
|3908787|                392|              392| FMY|  2016-04-21|          null|   PA|  3|  null|        2|     AT|
|5515827|                392|              392| DET|  2016-04-29|    2016-05-05|   PA| 64|     M|        1|     DL|
|2377315|                392|              392| NOG|  2016-04-13|          null|   AZ| 22|     F|        3|   null|
+-------+-------------------+-----------------+----+------------+--------------+-----+---+------+---------+-------+
only showing top 4 rows



In [155]:
df_demo_format.createOrReplaceTempView("demo")
df_states.createOrReplaceTempView("states")

df_demo.count()
spark.sql("""select count(*) from demo where state is  null or city is null""").show()
spark.sql("""select city,state from demo group by city,state having count(*)>1""").show()


+--------+
|count(1)|
+--------+
|       0|
+--------+

+----+-----+
|city|state|
+----+-----+
+----+-----+



In [159]:
df_airports_clean.createOrReplaceTempView("airports")
spark.sql("select count(*) from airports where local_code is null").show()


+----------+
|local_code|
+----------+
|      LL69|
|      1TS9|
|      3MI4|
|      K0Q5|
|       FHB|
|      XS29|
|       6X8|
|      TA29|
|       L11|
|      29NC|
|      86KS|
|      FL35|
|      3PA2|
|       EDC|
|      7MN5|
|      IN58|
|      07CT|
|      6OR8|
|      LA75|
|      OH91|
+----------+
only showing top 20 rows

+--------+
|count(1)|
+--------+
|       0|
+--------+



#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.